# Analiza dostupnog IXI skupa podataka

IXI skup podataka javno je dostupan skup podataka, a sadrži MR skenove odprilike 600 ispitanika.

Za svakog pacjenta dostupni su skenovi:
- T1
- T2
- PD
- MRA
- DTI

te metapodaci.

Podaci su priklupljeni iz tri bolnice (i tri MR skenera):
- Hammersmith Hospital (Philips 3T system)
- Guy’s Hospital (Philips 1.5T system)
- Institute of Psychiatry (GE 1.5T system)


In [1]:
#import knjiznica
import pandas as pd
import os
import re

In [2]:
# Učitaj metapodatke iz XLS datoteke
metadata_df = pd.read_excel('../dataset/raw_data/IXI.xls', sheet_name='Table')
ethnicity_df = pd.read_excel('../dataset/raw_data/IXI.xls', sheet_name='Ethnicity')
marital_status_df = pd.read_excel('../dataset/raw_data/IXI.xls', sheet_name='Marital Status')
ocupation_df = pd.read_excel('../dataset/raw_data/IXI.xls', sheet_name='Occupation')
qualification_df = pd.read_excel('../dataset/raw_data/IXI.xls', sheet_name='Qualification')
study_date_df = pd.read_excel('../dataset/raw_data/IXI.xls', sheet_name='Study Date')
metadata_df

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
0,1,1,170,80,2,3,5,2,1968-02-22,0,NaT,NaN
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824
...,...,...,...,...,...,...,...,...,...,...,...,...
614,652,1,163,80,1,1,1,5,1963-12-05,1,2006-12-01,42.989733
615,653,1,172,100,1,3,1,5,1960-09-14,1,2006-12-04,46.220397
616,655,1,165,72,1,1,1,1,1941-11-23,0,NaT,NaN
617,660,1,192,85,1,2,1,4,1941-04-12,0,NaT,NaN


In [3]:
#maknuti redove koji date_available je 0 jer to znaci da nisu dostupni skenovi
metadata_df = metadata_df[metadata_df['DATE_AVAILABLE'] != 0]
metadata_df

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824
5,15,1,181,90,2,1,6,5,1981-03-11,1,2005-06-23,24.284736
...,...,...,...,...,...,...,...,...,...,...,...,...
611,648,1,193,120,1,1,6,4,1959-03-08,1,2006-11-27,47.723477
613,651,1,175,61,3,2,8,2,1956-07-09,1,2006-12-01,50.395619
614,652,1,163,80,1,1,1,5,1963-12-05,1,2006-12-01,42.989733
615,653,1,172,100,1,3,1,5,1960-09-14,1,2006-12-04,46.220397


In [4]:
#ukloni duplikate redova
metadata_df = metadata_df.drop_duplicates()
metadata_df

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824
5,15,1,181,90,2,1,6,5,1981-03-11,1,2005-06-23,24.284736
...,...,...,...,...,...,...,...,...,...,...,...,...
611,648,1,193,120,1,1,6,4,1959-03-08,1,2006-11-27,47.723477
613,651,1,175,61,3,2,8,2,1956-07-09,1,2006-12-01,50.395619
614,652,1,163,80,1,1,1,5,1963-12-05,1,2006-12-01,42.989733
615,653,1,172,100,1,3,1,5,1960-09-14,1,2006-12-04,46.220397


In [5]:
print('Među ispitanicima postoje sljedeće etničke skupine:')
for index, row in ethnicity_df.iterrows():
    print(f'\t{row['ID']} - {row['ETHNIC']}')

Među ispitanicima postoje sljedeće etničke skupine:
	1 - White
	4 - Black or Black British
	3 - Asian or Asian British
	5 - Chinese
	6 - Other


In [6]:
print('Među ispitanicima postoje sljedeći bračni statusi:')
for index, row in marital_status_df.iterrows():
    print(f'\t{row['ID']} - {row['MARITAL']}')


Među ispitanicima postoje sljedeći bračni statusi:
	1 - Single
	2 - Married
	4 - Divorced or Separated
	3 - Cohabiting
	5 - Widowed


In [7]:
print('Među ispitanicima postoje sljedeći zanimanja:')
for index, row in ocupation_df.iterrows():
    print(f'\t{row['ID']} - {row['OCCUPATION']}')

Među ispitanicima postoje sljedeći zanimanja:
	1 - Go out to full time employment
	2 - Go out to part time employment (<25hrs)
	3 - Study at college or university
	4 - Full-time housework
	5 - Retired
	6 - Unemployed
	7 - Work for pay at home
	8 - Other


In [8]:
print('Među ispitanicima postoje sljedeće kvalifikacije:')
for index, row in qualification_df.iterrows():
    print(f'\t{row['ID']} - {row['QUALIFICATION']}')

Među ispitanicima postoje sljedeće kvalifikacije:
	1 - No qualifications
	2 - O-levels, GCSEs, or CSEs
	3 - A-levels
	4 - Further education e.g. City & Guilds / NVQs
	5 - University or Polytechnic degree


## Micanje podataka o ispitanicima čiji skenovi nisu dostupni i koje nećemo koristiti

In [19]:
#broj skenova u direktoriju
path_to_T1 = '../dataset/raw_data/IXI-T1'
path_to_T2 = '../dataset/raw_data/IXI-T2'

#broj skenova u direktoriju
T1_files = os.listdir(path_to_T1)
T2_files = os.listdir(path_to_T2)

In [20]:
print(f'U direktoriju {path_to_T1} se nalazi {len(T1_files)} skenova')
print(f'U direktoriju {path_to_T2} se nalazi {len(T2_files)} skenova')

U direktoriju ../dataset/raw_data/IXI-T1 se nalazi 581 skenova
U direktoriju ../dataset/raw_data/IXI-T2 se nalazi 578 skenova


In [21]:
t1_sken_ids = set(re.findall(r'IXI(\d+)', fname)[0] for fname in T1_files if re.search(r'IXI\d+', fname))

t2_sken_ids = set(re.findall(r'IXI(\d+)', fname)[0] for fname in T2_files if re.search(r'IXI\d+', fname))

metadata_ids = set(metadata_df['IXI_ID'].astype(str).str.zfill(3))

In [22]:
t1_skenovi_bez_metapodataka = t1_sken_ids - metadata_ids
t2_skenovi_bez_metapodataka = t2_sken_ids - metadata_ids

t1_bez_t2 = t1_sken_ids - t2_sken_ids
t2_bez_t1 = t2_sken_ids - t1_sken_ids

print(f'U direktoriju {path_to_T1} se nalazi {len(t1_skenovi_bez_metapodataka)} skenova bez metapodataka:\n {t1_skenovi_bez_metapodataka}')
print(f'U direktoriju {path_to_T2} se nalazi {len(t2_skenovi_bez_metapodataka)} skenova bez metapodataka:\n {t2_skenovi_bez_metapodataka}')
print(f'U direktoriju {path_to_T1} se nalazi {len(t1_bez_t2)} skenova koji nemaju odgovarajući sken u direktoriju {path_to_T2}:\n {t1_bez_t2}')
print(f'U direktoriju {path_to_T2} se nalazi {len(t2_bez_t1)} skenova koji nemaju odgovarajući sken u direktoriju {path_to_T1}:\n {t2_bez_t1}')

U direktoriju ../dataset/raw_data/IXI-T1 se nalazi 15 skenova bez metapodataka:
 {'081', '661', '088', '337', '117', '589', '457', '643', '228', '638', '333', '637', '345', '347', '340'}
U direktoriju ../dataset/raw_data/IXI-T2 se nalazi 15 skenova bez metapodataka:
 {'081', '661', '088', '337', '117', '589', '457', '643', '228', '638', '333', '637', '345', '347', '340'}
U direktoriju ../dataset/raw_data/IXI-T1 se nalazi 4 skenova koji nemaju odgovarajući sken u direktoriju ../dataset/raw_data/IXI-T2:
 {'116', '309', '182', '500'}
U direktoriju ../dataset/raw_data/IXI-T2 se nalazi 1 skenova koji nemaju odgovarajući sken u direktoriju ../dataset/raw_data/IXI-T1:
 {'580'}


In [23]:
#izbaci metapodatke za ispitanike koji nemaju skenove u oba direktorija
metadata_df = metadata_df[metadata_df['IXI_ID'].astype(str).str.zfill(3).isin(t1_sken_ids & t2_sken_ids)]
metadata_df

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824
5,15,1,181,90,2,1,6,5,1981-03-11,1,2005-06-23,24.284736
...,...,...,...,...,...,...,...,...,...,...,...,...
611,648,1,193,120,1,1,6,4,1959-03-08,1,2006-11-27,47.723477
613,651,1,175,61,3,2,8,2,1956-07-09,1,2006-12-01,50.395619
614,652,1,163,80,1,1,1,5,1963-12-05,1,2006-12-01,42.989733
615,653,1,172,100,1,3,1,5,1960-09-14,1,2006-12-04,46.220397


In [24]:
#pogled na duplikate po IXI_ID
metadata_df[metadata_df.duplicated(subset='IXI_ID', keep=False)]


,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
195,219,2,170,104,1,2,1,4,1953-02-08,1,2006-03-10,53.081451
196,219,2,164,73,1,4,1,5,1948-02-12 00:00:00,1,2006-03-10,58.072553
212,237,2,167,0,1,5,5,5,1928-06-11,1,2006-08-25,78.203970
213,237,2,0,0,0,0,0,0,1928-06-11,1,2006-08-25,78.203970
225,251,2,167,74,1,5,5,1,1926-06-28,1,2006-08-29,80.169747
226,251,2,0,0,0,0,0,0,1926-06-28,1,2006-08-29,80.169747
299,328,2,155,70,1,2,5,3,1944-11-03,1,2006-08-17,61.785079
300,328,1,0,0,0,0,0,0,1944-07-17,1,2006-08-17,62.083504
324,360,2,165,64,1,3,6,4,1952-01-27,1,2006-04-07,54.193018
325,360,2,0,0,0,0,0,0,1952-01-27,1,2006-04-07,54.193018


Iz ovoga možemo zaključiti da za većinu duplih zapisa samo treba zadržati prvi zapis jer je drugi skoro identičan, ali sa mnogo nedostajućih podataka.

Pacijenti koje ćemo detaljnije pogledati su oni s IXI_ID 219, 328, 416, 560.

In [25]:
#provjera za ispitanike: 219, 328, 416, 560
metadata_df[metadata_df['IXI_ID'].astype(str).isin(['219', '328', '416', '560'])]

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
195,219,2,170,104,1,2,1,4,1953-02-08,1,2006-03-10,53.081451
196,219,2,164,73,1,4,1,5,1948-02-12 00:00:00,1,2006-03-10,58.072553
299,328,2,155,70,1,2,5,3,1944-11-03,1,2006-08-17,61.785079
300,328,1,0,0,0,0,0,0,1944-07-17,1,2006-08-17,62.083504
382,416,2,1520,51,1,5,5,5,1934-07-03,1,2006-08-25,72.145106
383,416,2,176,102,0,0,0,0,1934-07-03,1,2006-08-25,72.145106
534,560,1,1850,80,1,1,5,3,1942-10-29,1,2006-09-11,63.868583
535,560,1,0,0,0,0,0,0,1942-10-29,1,2006-09-11,63.868583


Kod ispitanika 219 ostavljamo većinu prvog zapis. Premda su oba moguća, samo prvi zapis ima zapis rođenja u identičnom formatu kao i svi ostali ispitanici pa ćemo pretpostaviti da su to ispravne informacije. Visinu, težinu, bračni status i kvalifikacije ćemo popstaviti na 0.

Za ispitanika 328 imamo problem da se spol i datum rođenja razlikuju, ali kod drugog zapisa nema ostalih podataka te ćemo zato ponovo ostaviti samo prvi zapis uz postavljanje spola na 0.

Ispitanik 416 ima potpuno drugačije informacije o težini i visini, ali u drugom zapisu ponovno nedostaju podaci o kvalifikaciji, zanimanju, etnicitetu itd. pa ostavljamo prvi zapis uz visinu i težinu postavljeno na 0.

Ispitanik 560 ima mnogo nedostajućih informacija u drugom zapisu pa ostavljamo prvi uz postavljanje visine i težine na 0.

In [26]:
#zadrzi samo prvi redak duplikata
metadata_df_clean= metadata_df.drop_duplicates(subset='IXI_ID', keep='first')
metadata_df_clean

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824
5,15,1,181,90,2,1,6,5,1981-03-11,1,2005-06-23,24.284736
...,...,...,...,...,...,...,...,...,...,...,...,...
611,648,1,193,120,1,1,6,4,1959-03-08,1,2006-11-27,47.723477
613,651,1,175,61,3,2,8,2,1956-07-09,1,2006-12-01,50.395619
614,652,1,163,80,1,1,1,5,1963-12-05,1,2006-12-01,42.989733
615,653,1,172,100,1,3,1,5,1960-09-14,1,2006-12-04,46.220397


In [27]:
#ispravak visine ispitanika 416 i 560
metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '219', 'HEIGHT'] = 0
metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '219', 'WEIGHT'] = 0
metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '219', 'MARITAL_ID'] = 0
metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '219', 'QUALIFICATION_ID'] = 0

metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '328', 'SEX_ID (1=m, 2=f)'] = 0

metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '416', 'HEIGHT'] = 0
metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '416', 'WEIGHT'] = 0

metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '560', 'HEIGHT'] = 0
metadata_df_clean.loc[metadata_df_clean['IXI_ID'].astype(str) == '560', 'WEIGHT'] = 0

In [28]:
#preimenovanje stupaca SEX_ID (1=m, 2=f) u SEX_ID
metadata_df_clean = metadata_df_clean.rename(columns={'SEX_ID (1=m, 2=f)':'SEX_ID'})
#spremanje novih metapodataka u csv
metadata_df_clean.to_csv('../dataset/data/IXI_metadata.csv', index=False)